In [1]:
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import keras
import glob
import json

from matplotlib.pyplot import specgram
import matplotlib.pyplot as plt
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.models import Model
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from librosa.util import lazy

In [2]:
mylist= os.listdir('RawData/')
mylist

['03-01-01-01-01-01-01.wav',
 '03-01-01-01-01-01-02.wav',
 '03-01-01-01-01-01-03.wav',
 '03-01-01-01-01-01-04.wav',
 '03-01-01-01-01-01-05.wav',
 '03-01-01-01-01-01-06.wav',
 '03-01-01-01-01-01-07.wav',
 '03-01-01-01-01-01-08.wav',
 '03-01-01-01-01-01-09.wav',
 '03-01-01-01-01-01-10.wav',
 '03-01-01-01-01-01-11.wav',
 '03-01-01-01-01-01-12.wav',
 '03-01-01-01-01-01-13.wav',
 '03-01-01-01-01-01-14.wav',
 '03-01-01-01-01-01-15.wav',
 '03-01-01-01-01-01-16.wav',
 '03-01-01-01-01-01-17.wav',
 '03-01-01-01-01-01-18.wav',
 '03-01-01-01-01-01-19.wav',
 '03-01-01-01-01-01-20.wav',
 '03-01-01-01-01-01-21.wav',
 '03-01-01-01-01-01-22.wav',
 '03-01-01-01-01-01-23.wav',
 '03-01-01-01-01-01-24.wav',
 '03-01-01-01-01-02-01.wav',
 '03-01-01-01-01-02-02.wav',
 '03-01-01-01-01-02-03.wav',
 '03-01-01-01-01-02-04.wav',
 '03-01-01-01-01-02-05.wav',
 '03-01-01-01-01-02-06.wav',
 '03-01-01-01-01-02-07.wav',
 '03-01-01-01-01-02-08.wav',
 '03-01-01-01-01-02-09.wav',
 '03-01-01-01-01-02-10.wav',
 '03-01-01-01-

In [3]:
print(mylist[1439])
print(mylist[1151][6:-16])
print(mylist[1151][18:-4])
print(mylist[3333])

03-01-08-02-02-02-24.wav
07
24
YAF_jug_happy.wav


In [4]:
feeling_list=[]
for item in mylist:
    if item[0].isnumeric():
        if item[6:-16]=='01' and int(item[18:-4])%2==0:
            feeling_list.append('female_neutral')
        elif item[6:-16]=='01' and int(item[18:-4])%2==1:
            feeling_list.append('male_neutral')
        
        # Asignaremos la categoría "calm" como "neutral"
        elif item[6:-16]=='02' and int(item[18:-4])%2==0:
            feeling_list.append('female_neutral')
        elif item[6:-16]=='02' and int(item[18:-4])%2==1:
            feeling_list.append('male_neutral')
        
        elif item[6:-16]=='03' and int(item[18:-4])%2==0:
            feeling_list.append('female_happy')
        elif item[6:-16]=='03' and int(item[18:-4])%2==1:
            feeling_list.append('male_happy')
        
        elif item[6:-16]=='04' and int(item[18:-4])%2==0:
            feeling_list.append('female_sad')
        elif item[6:-16]=='04' and int(item[18:-4])%2==1:
            feeling_list.append('male_sad')
        
        elif item[6:-16]=='05' and int(item[18:-4])%2==0:
            feeling_list.append('female_angry')
        elif item[6:-16]=='05' and int(item[18:-4])%2==1:
            feeling_list.append('male_angry')
        
        elif item[6:-16]=='06' and int(item[18:-4])%2==0:
            feeling_list.append('female_fearful')
        elif item[6:-16]=='06' and int(item[18:-4])%2==1:
            feeling_list.append('male_fearful')
        
        elif item[6:-16]=='07' and int(item[18:-4])%2==0:
            feeling_list.append('female_disgust')
        elif item[6:-16]=='07' and int(item[18:-4])%2==1:
            feeling_list.append('male_disgust')
        
        elif item[6:-16]=='08' and int(item[18:-4])%2==0:
            feeling_list.append('female_surprised')
        elif item[6:-16]=='08' and int(item[18:-4])%2==1:
            feeling_list.append('male_surprised')
    else:
        # Extraer ZZZ del nombre de archivo con formato XXX_YYY_ZZZ.wav
        partes = item.split('_')  # Dividir el nombre de archivo por '_'
        if len(partes) == 3 and len(partes[0]) == 3:  # Verificar el formato del nombre de archivo
            zzz = partes[2]  # Extraer la parte ZZZ
        else:
            zzz = ""  # Establecer zzz como cadena vacía si el formato no coincide
        
        if zzz == 'fear':
            zzz = 'fearful'
        
        if zzz == 'ps':
            zzz = 'surprised'
        
        feeling_list.append(f"female_{zzz}")  # Añadir la categoría de sentimiento a la lista

In [5]:
labels = pd.DataFrame(feeling_list)
labels[:1000]

,0
0,male_neutral
1,female_neutral
2,male_neutral
3,female_neutral
4,male_neutral
...,...
995,female_fearful
996,male_fearful
997,female_fearful
998,male_fearful


## Getting the features of audio files using librosa

In [6]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    X, sample_rate = librosa.load('RawData/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    feature = mfccs
    df.loc[bookmark] = [feature]
    bookmark=bookmark+1        

In [7]:
df[:20]

,feature
0,"[-65.801285, -65.801285, -65.801285, -65.80128..."
1,"[-63.214157, -63.214157, -63.214157, -63.21415..."
2,"[-60.258987, -60.258987, -60.258987, -60.25898..."
3,"[-66.48846, -66.48846, -66.48846, -66.48846, -..."
4,"[-62.027878, -60.52594, -64.739456, -66.27164,..."
5,"[-70.56438, -69.67404, -68.959564, -66.6732, -..."
6,"[-66.11158, -65.471504, -65.54894, -65.61661, ..."
7,"[-59.08398, -54.093624, -49.72882, -51.27526, ..."
8,"[-45.715374, -47.650826, -50.377445, -50.83738..."
9,"[-53.32299, -52.167725, -51.030327, -51.41492,..."


In [8]:
df2 = pd.DataFrame(df['feature'].values.tolist())

In [9]:
newdf = pd.concat([df2,labels], axis=1)

In [10]:
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf[:30]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
0,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,-65.801285,...,-62.295700,-64.930496,-65.332520,-64.117332,-64.320320,-64.971794,-63.156208,-64.258797,-64.468193,male_neutral
1,-63.214157,-63.214157,-63.214157,-63.214157,-63.214157,-63.167240,-63.385445,-63.461830,-63.336113,-63.204216,...,-53.428989,-53.139889,-52.854271,-55.248135,-57.682434,-57.166489,-57.040077,-56.105301,-59.386353,female_neutral
2,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,-60.258987,...,-50.511784,-49.082012,-47.701584,-48.795418,-52.424294,-54.828476,-52.233757,-49.679409,-49.952641,male_neutral
3,-66.488457,-66.488457,-66.488457,-66.488457,-66.488457,-66.488457,-66.527412,-66.554977,-66.534813,-66.488457,...,-62.122272,-64.444450,-65.315735,-65.285492,-65.365860,-64.971207,-64.157776,-64.913437,-66.488457,female_neutral
4,-62.027878,-60.525940,-64.739456,-66.271637,-61.207905,-60.241585,-62.637955,-63.144920,-60.527340,-59.102253,...,-54.736412,-50.871803,-51.928799,-56.389347,-57.153381,-54.602703,-54.804359,-60.364048,-64.864937,male_neutral
5,-70.564377,-69.674042,-68.959564,-66.673203,-65.417709,-65.465439,-65.583076,-64.787086,-63.228130,-64.657501,...,-66.148758,-61.432064,-60.566154,-62.682816,-62.865822,-61.645000,-60.380146,-59.267979,-61.693340,female_neutral
6,-66.111580,-65.471504,-65.548943,-65.616608,-65.774040,-65.003319,-64.079361,-63.866074,-63.786770,-64.031166,...,-28.006758,-27.373323,-28.532036,-28.788694,-30.285656,-30.725269,-32.715199,-34.577053,-35.818699,male_neutral
7,-59.083981,-54.093624,-49.728821,-51.275261,-51.939510,-52.804752,-54.459030,-52.764038,-52.708351,-50.039146,...,-51.501465,-52.597069,-52.922337,-52.165787,-53.210182,-55.510208,-55.963039,-54.293941,-51.897617,female_neutral
8,-45.715374,-47.650826,-50.377445,-50.837379,-51.157372,-47.423889,-43.265408,-40.213829,-39.601009,-39.982784,...,-54.270748,-52.245163,-53.859722,-58.266106,-55.575455,-54.526115,-53.132980,-53.899075,-55.315193,male_neutral
9,-53.322990,-52.167725,-51.030327,-51.414921,-51.138325,-50.835449,-50.257061,-49.667683,-49.851974,-50.756752,...,-48.854332,-47.892612,-47.630802,-47.916912,-47.281517,-46.504807,-46.667953,-49.411640,-51.124687,female_neutral


In [11]:
rnewdf = shuffle(newdf)
rnewdf[:30]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
525,-59.905571,-58.336311,-57.889641,-58.005428,-57.755058,-56.624138,-55.128700,-53.701946,-54.891453,-55.263908,...,-53.670994,-53.048256,-51.910339,-53.679409,-55.204105,-54.431973,-54.878006,-56.326054,-58.208088,female_sad
4192,-18.509506,-21.108891,-33.342205,-31.673998,-33.476208,-33.535194,-36.944862,-36.376335,-35.262142,-37.342045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_disgust.wav
3467,-13.262000,-15.657599,-25.296511,-24.334513,-23.451796,-23.979546,-23.804674,-24.889362,-23.692118,-24.381742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_neutral.wav
129,-58.702705,-54.495502,-51.199612,-50.520729,-52.475204,-53.485710,-50.421196,-50.866367,-52.749458,-52.686340,...,-48.035141,-45.780640,-46.927269,-49.604942,-53.928097,-52.472641,-49.343410,-49.750717,-48.225830,female_neutral
3502,-17.359293,-19.429991,-22.378592,-22.373472,-22.069815,-23.404091,-24.974937,-24.382305,-22.650835,-21.296974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_neutral.wav
3417,-17.644753,-19.657127,-27.349201,-22.955301,-20.903080,-21.141438,-23.947794,-22.819784,-23.926210,-24.008131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_happy.wav
2591,-24.511415,-25.266907,-28.580212,-27.133480,-27.805859,-28.262867,-27.337944,-27.776289,-26.972799,-26.914541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_neutral.wav
3575,-14.970184,-19.038898,-27.353018,-25.599506,-24.680025,-23.799568,-26.515554,-29.197752,-32.685543,-34.825153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_angry.wav
3108,0.753879,0.392685,-5.429655,-9.233579,-11.733248,-15.474287,-17.755287,-20.970478,-22.784386,-26.773487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_fear.wav
2893,-19.504620,-22.258101,-26.665161,-27.146471,-28.003286,-28.146646,-27.411446,-23.774502,-23.244101,-26.030113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_neutral.wav


In [12]:
rnewdf=rnewdf.fillna(0)
rnewdf

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
525,-59.905571,-58.336311,-57.889641,-58.005428,-57.755058,-56.624138,-55.128700,-53.701946,-54.891453,-55.263908,...,-53.670994,-53.048256,-51.910339,-53.679409,-55.204105,-54.431973,-54.878006,-56.326054,-58.208088,female_sad
4192,-18.509506,-21.108891,-33.342205,-31.673998,-33.476208,-33.535194,-36.944862,-36.376335,-35.262142,-37.342045,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_disgust.wav
3467,-13.262000,-15.657599,-25.296511,-24.334513,-23.451796,-23.979546,-23.804674,-24.889362,-23.692118,-24.381742,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_neutral.wav
129,-58.702705,-54.495502,-51.199612,-50.520729,-52.475204,-53.485710,-50.421196,-50.866367,-52.749458,-52.686340,...,-48.035141,-45.780640,-46.927269,-49.604942,-53.928097,-52.472641,-49.343410,-49.750717,-48.225830,female_neutral
3502,-17.359293,-19.429991,-22.378592,-22.373472,-22.069815,-23.404091,-24.974937,-24.382305,-22.650835,-21.296974,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_neutral.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225,-8.939459,-13.455708,-26.001446,-24.658155,-23.354698,-22.874725,-23.062094,-20.648821,-16.817427,-16.414856,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_sad.wav
3931,-6.231580,-10.259149,-23.090925,-23.315044,-23.478123,-21.506664,-21.256479,-21.269041,-23.357466,-24.466553,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_sad.wav
536,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-73.914192,...,-54.466328,-53.319332,-54.592472,-57.332699,-56.585026,-59.388550,-56.331051,-55.907162,-57.657578,male_sad
4059,-11.940023,-16.150621,-31.977661,-29.221764,-23.870867,-22.619938,-23.838047,-23.190052,-24.023460,-24.788576,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_disgust.wav


## Dividing the data into test and train

In [13]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

In [14]:
train[250:260]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
463,-53.479824,-53.717781,-55.004333,-54.965492,-54.323532,-55.139442,-51.769375,-51.738560,-51.537983,-50.650997,...,-40.021450,-39.736160,-40.806484,-42.065731,-41.077335,-40.920578,-43.015312,-44.794868,-43.672207,female_happy
3019,-11.979396,-15.452549,-21.303932,-20.298740,-19.774040,-21.004421,-21.372593,-21.504307,-22.626352,-22.991306,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_neutral.wav
2178,-25.459606,-27.112450,-29.656590,-28.337618,-27.266666,-27.366936,-28.118233,-29.273787,-30.821232,-30.855011,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_neutral.wav
351,-53.681492,-52.698265,-52.419876,-52.407406,-52.703842,-52.704060,-52.881699,-53.652988,-53.442471,-53.400349,...,-48.523659,-51.101254,-53.086731,-50.750832,-51.106731,-53.263645,-53.704834,-53.511917,-53.947990,female_happy
872,-80.181229,-76.616661,-77.295746,-74.925789,-75.429192,-78.949661,-81.305908,-81.305908,-81.305908,-81.305908,...,-59.699242,-57.292263,-57.995979,-60.571743,-55.889561,-55.076279,-55.283501,-55.467854,-58.983982,male_fearful
1491,-23.639774,-23.477354,-24.091169,-23.420595,-23.338327,-23.435211,-22.110514,-22.453173,-23.275908,-24.163776,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_happy.wav
3188,-30.522551,-29.984688,-30.899879,-31.829315,-33.469425,-36.711483,-37.743416,-36.237125,-33.180744,-32.672516,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_ps.wav
494,-39.693199,-41.224266,-47.779995,-47.868202,-48.521194,-49.263634,-50.616024,-50.911629,-50.581928,-52.067360,...,-49.006645,-49.691952,-50.795109,-51.055195,-53.019577,-48.177029,-47.039829,-49.851402,-50.915058,male_sad
2614,-19.643322,-22.722767,-31.850615,-31.628971,-31.272562,-31.227476,-32.025898,-31.115969,-31.593550,-32.244888,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,female_sad.wav
917,-63.375122,-61.009460,-60.393532,-60.479393,-61.317410,-64.020103,-60.963615,-60.685181,-62.826859,-62.177471,...,-58.343952,-61.135880,-60.476467,-57.457504,-55.493618,-56.426872,-60.389137,-63.704323,-61.368813,female_fearful


In [15]:
trainfeatures = train.iloc[:, :-1]
trainfeatures

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
525,-59.905571,-58.336311,-57.889641,-58.005428,-57.755058,-56.624138,-55.128700,-53.701946,-54.891453,-55.263908,...,-51.793133,-53.670994,-53.048256,-51.910339,-53.679409,-55.204105,-54.431973,-54.878006,-56.326054,-58.208088
4192,-18.509506,-21.108891,-33.342205,-31.673998,-33.476208,-33.535194,-36.944862,-36.376335,-35.262142,-37.342045,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3467,-13.262000,-15.657599,-25.296511,-24.334513,-23.451796,-23.979546,-23.804674,-24.889362,-23.692118,-24.381742,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
129,-58.702705,-54.495502,-51.199612,-50.520729,-52.475204,-53.485710,-50.421196,-50.866367,-52.749458,-52.686340,...,-49.505489,-48.035141,-45.780640,-46.927269,-49.604942,-53.928097,-52.472641,-49.343410,-49.750717,-48.225830
3502,-17.359293,-19.429991,-22.378592,-22.373472,-22.069815,-23.404091,-24.974937,-24.382305,-22.650835,-21.296974,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460,-12.567047,-15.225738,-24.537479,-24.705811,-21.979412,-21.353817,-21.484394,-21.782078,-23.788609,-23.558516,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3931,-6.231580,-10.259149,-23.090925,-23.315044,-23.478123,-21.506664,-21.256479,-21.269041,-23.357466,-24.466553,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
536,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-74.321205,-73.914192,...,-53.823330,-54.466328,-53.319332,-54.592472,-57.332699,-56.585026,-59.388550,-56.331051,-55.907162,-57.657578
4059,-11.940023,-16.150621,-31.977661,-29.221764,-23.870867,-22.619938,-23.838047,-23.190052,-24.023460,-24.788576,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
trainlabel = train.iloc[:, -1:]
trainlabel

,0
525,female_sad
4192,female_disgust.wav
3467,female_neutral.wav
129,female_neutral
3502,female_neutral.wav
...,...
3460,female_neutral.wav
3931,female_sad.wav
536,male_sad
4059,female_disgust.wav


In [17]:
testfeatures = test.iloc[:, :-1]
testfeatures

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
3575,-14.970184,-19.038898,-27.353018,-25.599506,-24.680025,-23.799568,-26.515554,-29.197752,-32.685543,-34.825153,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
265,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,-65.190414,...,-33.761417,-31.972822,-31.609589,-30.257240,-30.320049,-29.578274,-29.647501,-29.809294,-29.711689,-28.573994
928,-22.277245,-25.433439,-36.630814,-37.152706,-37.555515,-37.238705,-38.051785,-37.925919,-37.066647,-38.827522,...,-41.725491,-39.768753,-40.304504,-41.926205,-41.441574,-42.533531,-44.090847,-43.533714,-32.996002,-25.445312
2883,-20.505239,-23.809855,-34.440861,-34.267540,-31.686153,-31.514376,-29.980227,-28.688011,-26.857334,-27.143250,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
818,-39.460392,-42.556416,-45.279720,-45.535896,-45.075863,-45.133282,-44.699341,-45.068985,-45.376606,-45.128670,...,-17.034382,-16.539120,-15.113464,-18.174229,-21.630623,-22.855438,-21.752399,-21.372484,-18.433481,-10.575611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2721,-17.257875,-18.891880,-26.324028,-26.852053,-29.033165,-31.887133,-32.413368,-29.306637,-29.049173,-26.742109,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2722,-12.849527,-17.429958,-31.230539,-31.052521,-32.057018,-33.057541,-31.433859,-30.903414,-29.776693,-30.546375,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4126,-20.199116,-22.964903,-29.637764,-29.758801,-28.506399,-27.401659,-28.800831,-31.321653,-33.968575,-37.014999,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1886,-16.233179,-18.713621,-25.415550,-27.179594,-25.941484,-25.917362,-26.628784,-25.786156,-26.355938,-26.006203,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
testlabel = test.iloc[:, -1:]
testlabel

,0
3575,female_angry.wav
265,female_neutral
928,male_fearful
2883,female_disgust.wav
818,male_angry
...,...
2721,female_disgust.wav
2722,female_fear.wav
4126,female_ps.wav
1886,female_sad.wav


In [19]:
X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))

c:\Users\bianc\Desktop\SMF\Hack a Boss -  Data Sciences\VirtualEnv\mod3-data-science\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\bianc\Desktop\SMF\Hack a Boss -  Data Sciences\VirtualEnv\mod3-data-science\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
X_train.shape

(3375, 216)

## Changing dimension for CNN model

In [22]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [23]:
# Definir el modelo
model = Sequential()

# model.add(Conv1D(256, 5,padding='same',
#                  input_shape=(216,1)))
# model.add(Activation('relu'))
# model.add(Conv1D(128, 5,padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(MaxPooling1D(pool_size=(8)))
# model.add(Conv1D(128, 5,padding='same',))
# model.add(Activation('relu'))
# #model.add(Conv1D(128, 5,padding='same',))
# #model.add(Activation('relu'))
# #model.add(Conv1D(128, 5,padding='same',))
# #model.add(Activation('relu'))
# #model.add(Dropout(0.2))
# model.add(Conv1D(128, 5,padding='same',))
# model.add(Activation('relu'))
# model.add(Flatten())
# model.add(Dense(12))
# model.add(Activation('softmax'))

model.add(Conv1D(256, 5, padding='same', input_shape=(216, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(128, 5, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=8))
model.add(Conv1D(128, 5, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(128, 5, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(12))  # Asegúrate de que coincide con el número de clases
model.add(Activation('softmax'))

# Definir el optimizador correctamente
opt = RMSprop(learning_rate=0.00001, decay=1e-6)

c:\Users\bianc\Desktop\SMF\Hack a Boss -  Data Sciences\VirtualEnv\mod3-data-science\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\bianc\Desktop\SMF\Hack a Boss -  Data Sciences\VirtualEnv\mod3-data-science\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 216, 256)       │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 216, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 216, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 216, 128)       │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 216, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 216, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 216, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 27, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 27, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 27, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 27, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 27, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 27, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 27, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 27, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 27, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 27, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 27, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3456)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       442,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 128)            │             

 Total params: 859,276 (3.28 MB)

 Trainable params: 857,484 (3.27 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [25]:
# configures the model for training:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

### Removed the whole training part for avoiding unnecessary long epochs list

In [26]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size=32, epochs=1000, validation_data=(x_testcnn, y_test))
# cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=700, validation_data=(x_testcnn, y_test))

plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/1000


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 21), output.shape=(None, 12)

## Saving the model

In [ ]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at c:\Users\diego\Desktop\testnotebook\Speech-Emotion-Analyzer\saved_models\Emotion_Voice_Detection_Model.h5 


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

## Loading the model

In [ ]:
# loading json and creating model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Voice_Detection_Model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
compile_metrics: 52.80%


## Predicting emotions on the test data

In [ ]:
preds = loaded_model.predict(x_testcnn, 
                         batch_size=32, 
                         verbose=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [ ]:
preds

array([[2.0227712e-03, 9.6189308e-01, 1.7221543e-04, ..., 1.5175356e-05,
        6.7600857e-05, 4.8620910e-05],
       [1.2197958e-03, 4.4126543e-03, 1.0004833e-03, ..., 9.9739926e-03,
        1.6837782e-04, 2.1589111e-04],
       [1.6870309e-03, 9.8447776e-01, 9.1042617e-05, ..., 2.5419798e-04,
        8.0815982e-05, 7.4553187e-05],
       ...,
       [6.1123329e-04, 4.9028830e-03, 9.8370636e-01, ..., 7.5109267e-05,
        2.6878197e-04, 7.5201731e-04],
       [3.1410477e-01, 3.6658274e-04, 3.6705915e-02, ..., 1.1037292e-02,
        2.4490280e-03, 4.6654884e-03],
       [1.6414818e-04, 1.6775139e-05, 7.0504102e-05, ..., 1.3070037e-05,
        2.2725202e-05, 9.3921735e-05]], dtype=float32)

In [ ]:
preds1=preds.argmax(axis=1)

In [ ]:
preds1

array([ 1,  3,  1,  0,  4,  0,  9,  5, 10,  2,  3,  5,  7,  6,  0,  9,  8,
        6,  4,  2,  9, 11,  4,  3,  8,  8,  0,  8,  4,  8,  1,  3,  1,  4,
       10,  3,  0,  0,  4,  8,  4,  6,  1,  5,  8,  5,  3,  7,  5, 10,  2,
        9,  4,  1,  2,  9,  3,  1,  5,  5,  6,  8,  7, 10,  7,  7,  5,  5,
       10,  4,  5, 10,  2,  7,  9,  3,  0,  4,  2,  3, 10,  2,  6,  8,  0,
        1,  3,  9,  2,  4,  1,  2,  0,  8,  2,  0,  5,  6,  2,  6, 11,  9,
        5,  3,  2,  9,  0,  9,  5,  6, 11,  0,  8,  5,  2,  5, 10,  8,  8,
       11,  9,  2, 10,  3,  8,  1,  1,  5,  9,  6,  3,  6,  6, 11,  3,  6,
        6,  6,  9,  2,  1,  5,  3,  5,  6,  4,  5, 10,  5,  3,  8,  2,  9,
        5,  8,  9,  3,  7,  7,  9,  0,  0,  2,  0,  4,  0,  2,  1,  1,  8,
        7,  3,  4,  9,  3,  0,  2,  6, 11, 11,  7,  4, 10,  7,  8,  0,  1,
        1,  5, 10,  3, 10,  0,  1,  3,  0, 10,  3,  1,  7,  2,  4,  0,  5,
        5,  5,  1,  9,  9,  7,  9,  2,  3,  5], dtype=int64)

In [ ]:
abc = preds1.astype(int).flatten()

In [ ]:
predictions = (lb.inverse_transform((abc)))

In [ ]:
preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:20]

,predictedvalues
0,female_disgust
1,female_happy
2,female_disgust
3,female_angry
4,female_sad
5,female_angry
6,male_happy
7,female_surprised
8,male_sad
9,female_fearful


In [ ]:
actual=y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

In [ ]:
actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:20]

,actualvalues
0,female_sad
1,female_surprised
2,female_sad
3,female_surprised
4,female_sad
5,female_angry
6,male_happy
7,female_happy
8,male_sad
9,female_happy


In [ ]:
finaldf = actualdf.join(preddf)

## Actual v/s Predicted emotions

In [ ]:
finaldf[170:180]

,actualvalues,predictedvalues
170,male_disgust,male_disgust
171,female_happy,female_happy
172,female_disgust,female_sad
173,male_fearful,male_happy
174,male_sad,female_happy
175,female_angry,female_angry
176,female_fearful,female_fearful
177,male_angry,male_angry
178,male_surprised,male_surprised
179,male_surprised,male_surprised


In [ ]:
finaldf.groupby('actualvalues').count()

,predictedvalues
actualvalues,
female_angry,21
female_disgust,20
female_fearful,18
female_happy,19
female_sad,17
female_surprised,28
male_angry,15
male_disgust,17
male_fearful,9


In [ ]:
finaldf.groupby('predictedvalues').count()

,actualvalues
predictedvalues,
female_angry,21
female_disgust,19
female_fearful,21
female_happy,23
female_sad,16
female_surprised,26
male_angry,16
male_disgust,13
male_fearful,18


In [ ]:
finaldf.to_csv('Predictions.csv', index=False)